In [1]:
import numpy as np
import pandas as pd

In [2]:
stop_words = ['i',
'me',
'my',
'myself',
'we',
'our',
'ours',
'ourselves',
'you',
'your',
'yours',
'yourself',
'yourselves',
'he',
'him',
'his',
'himself',
'she',
'her',
'hers',
'herself',
'it',
'its',
'itself',
'they',
'them',
'their',
'theirs',
'themselves',
'what',
'which',
'who',
'whom',
'this',
'that',
'these',
'those',
'am',
'is',
'are',
'was',
'were',
'be',
'been',
'being',
'have',
'has',
'had',
'having',
'do',
'does',
'did',
'doing',
'a',
'an',
'the',
'and',
'but',
'if',
'or',
'because',
'as',
'until',
'while',
'of',
'at',
'by',
'for',
'with',
'about',
'against',
'between',
'into',
'through',
'during',
'before',
'after',
'above',
'below',
'to',
'from',
'up',
'down',
'in',
'out',
'on',
'off',
'over',
'under',
'again',
'further',
'then',
'once',
'here',
'there',
'when',
'where',
'why',
'how',
'all',
'any',
'both',
'each',
'few',
'more',
'most',
'other',
'some',
'such',
'no',
'nor',
'not',
'only',
'own',
'same',
'so',
'than',
'too',
'very',
'can',
'will',
'just',
'don',
'should',
'now']

In [3]:
df = pd.read_csv("spam_or_not_spam.csv")
print(df.email)
df=df.replace(np.nan,'hello how are you')
df.columns=['doc', 'class']
df = np.array(df)
x_data = df[:, 0] # 1st column

0        date wed NUMBER aug NUMBER NUMBER NUMBER NUMB...
1       martin a posted tassos papadopoulos the greek ...
2       man threatens explosion in moscow thursday aug...
3       klez the virus that won t die already the most...
4        in adding cream to spaghetti carbonara which ...
                              ...                        
2995     abc s good morning america ranks it the NUMBE...
2996     hyperlink hyperlink hyperlink let mortgage le...
2997     thank you for shopping with us gifts for all ...
2998     the famous ebay marketing e course learn to s...
2999     hello this is chinese traditional 子 件 NUMBER世...
Name: email, Length: 3000, dtype: object


In [4]:
def fit(x_train, y_train) :
    result = {}
    class_values = set(y_train)
    for curr in class_values :
        result[curr] = {}
        result["total_data"] = len(y_train)
        current_class_rows = (y_train == curr)
        x_train_current = x_train[current_class_rows]
        y_train_current = y_train[current_class_rows]
        
        feat_count = x_train.shape[1] # (rows, cols)
        result[curr]["total_count"] = len(y_train_current)
        for j in range(1, feat_count+1) :
            result[curr][j] = {}
            all_possible_values = set(x_train[:, j-1])
            for curr_value in all_possible_values :
                result[curr][j][curr_value] = (x_train_current[:, j-1] == curr_value).sum()
    return result

In [5]:
def probability(dictionary, x, curr_class) :
    output = np.log(dictionary[curr_class]["total_count"]) - np.log(dictionary["total_data"])
    num_features = len(dictionary[curr_class].keys()) - 1 # due to "total_count" key
    for j in range(1, num_features+1) :
        xj = x[j-1]
        curr_class_with_value_xj = dictionary[curr_class][j][xj] + 1 # laplace correction
        count_curr_class = dictionary[curr_class]["total_count"] + len(dictionary[curr_class][j].keys())
        curr_xj_prob = np.log(curr_class_with_value_xj) - np.log(count_curr_class)
        output = output + curr_xj_prob
    return output

In [6]:
def predictSinglePoint(dictionary, x) :
    classes = dictionary.keys() # setosa, virgin.., versi, "total_data"
    best_prob = -1
    best_class = -1
    firstrun = True
    for curr_class in classes :
        if(curr_class == "total_data") :
            continue
        p_curr_class = probability(dictionary, x, curr_class) # dictionary[curr_class]["total_count"]
        if(firstrun or p_curr_class > best_prob) :
            best_prob = p_curr_class
            best_class = curr_class
        firstrun = False
    return best_class

In [7]:
def predict(dictionary, x_test) :
    y_pred = []
    for x in x_test :
        x_class = predictSinglePoint(dictionary, x)
        y_pred.append(x_class)
    return y_pred

In [8]:
diction = {}
k = 100
for it in x_data :
    currlist = it.split(" ")
    for itr in currlist :
        if(itr in stop_words) :
            continue
        if(itr in diction.keys()) :
            diction[itr] += 1
        else :
            diction[itr] = 1

diction = dict(sorted(diction.items(), key=lambda item: item[1], reverse=True))
features = dict(list(diction.items())[0: k])
feat_keys = list(features.keys())

idx = {}
for i in range(0, len(feat_keys)) :
    idx[feat_keys[i]] = i


x_train = []

for it in x_data :
    currlist = (it.split(" "))
    curr_x_train = [0]*100
    for itr in feat_keys :
        curr_x_train[idx[itr]] = currlist.count(itr)
    x_train.append(curr_x_train)

x_train = np.array(x_train)

In [9]:
print(x_train.shape)

(3000, 100)


In [10]:
y_train = list(df[:, 1])
y_train = np.array(y_train)
print(y_train.shape[0])

3000


In [11]:
dictionary = fit(x_train, y_train)

In [12]:
y_pred = predict(dictionary, x_train)
y_pred

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [13]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_train, y_pred))

[[2451   49]
 [  80  420]]


In [14]:
from sklearn.metrics import accuracy_score 
accuracy_score (y_train,y_pred)

0.957